#**Spring 2025 BAA1030 Data Analytics & Story Telling (20074)**

Kapil Devkant Gupta

A00011783

Assignment 2: Quarto Report


# **Wealth, Health, and Gender: A Global Look at Adolescent Inactivity and Child Deprivation**

#**Introduction**
This dashboard explores the interconnectedness of economic prosperity, adolescent health, and gender inequality around the world. Using real-world data from UNICEF, we uncover how wealth shapes — but does not fully determine — childhood deprivation and physical inactivity patterns, especially among adolescent girls.The story progresses from highlighting the worst-affected countries to analyzing long-term trends and gender disparities.

In [ ]:
!pip install plotly


In [ ]:
# Imports
import polars as pl
from plotnine import *
import matplotlib.pyplot as plt
import geopandas as gpd
import plotly.express as px

indicator_1 = pl.read_csv('/content/Contents/unicef_indicator_1 (1).csv')
indicator_2 = pl.read_csv('/content/Contents/unicef_indicator_2 (1).csv')
metadata = pl.read_csv(
    '/content/Contents/unicef_metadata (1).csv',
    infer_schema_length=10000,
    try_parse_dates=True,
    schema_overrides={"Population, total": pl.Float64}
)

In [ ]:
import polars as pl

# 1️⃣ Preprocess: Top 10 countries with highest adolescent physical inactivity (latest year)
inactivity_female = indicator_1.filter(pl.col("sex") == "Female")

inactivity_latest = (
    inactivity_female.sort("time_period")
    .group_by("country")
    .agg(pl.all().last())
)

top10_inactive = (
    inactivity_latest.sort("obs_value", descending=True)
    .head(10)
)

# 2️⃣ Preprocess for Scatterplot (Deprivation vs GDP)

deprivation_recent = (
    indicator_2.filter(pl.col("sex") == "Total")
    .sort("time_period")
    .group_by("country")
    .agg(pl.all().last())
)

# Merge deprivation with GDP from metadata
# Note: in polars, we use join not merge
deprivation_gdp = (
    deprivation_recent.join(
        metadata.select(["country", "year", "GDP per capita (constant 2015 US$)"]),
        left_on=["country", "time_period"],
        right_on=["country", "year"],
        how="inner"
    )
    .drop_nulls(["GDP per capita (constant 2015 US$)", "obs_value"])
)

# Rename GDP column
deprivation_gdp = deprivation_gdp.rename({"GDP per capita (constant 2015 US$)": "gdp_per_capita"})

# 3️⃣ Preprocess: Time series data for selected countries
selected_countries = ["India", "Brazil", "United States"]

time_series_data = indicator_1.filter(
    (pl.col("country").is_in(selected_countries)) &
    (pl.col("sex") == "Female")
)

# 4️⃣ Preprocess: Boxplot gender comparison
inactivity_latest_gender = (
    indicator_1.sort("time_period")
    .group_by(["country", "sex"])
    .agg(pl.all().last())
    .drop_nulls(["obs_value"])
)


## **1. Top 10 Countries with Female Adolescent Inactivity**
"*Where Girls Are Falling Behind: The Global Hotspots of Adolescent Inactivity"*


**Insight**

A small group of countries faces alarmingly high rates of female adolescent inactivity.
This signals deeper social, cultural, or infrastructural challenges that limit active lifestyles for girls.
Targeted interventions in these hotspots could dramatically improve global adolescent health outcomes.

**Visualization**

Horizontal Bar Chart showing the top 10 countries ranked by female adolescent inactivity rates.



In [ ]:

top10_plot = (
    ggplot(top10_inactive, aes(x='reorder(country, obs_value)', y='obs_value'))
    + geom_bar(stat='identity', fill='skyblue')
    + coord_flip()
    + labs(
        title='Countries with Highest Female Adolescent Inactivity',
        x='Country',
        y='Inactivity (%)'
    )
    + theme_minimal()
    + theme(
        figure_size=(10, 6),
        plot_title=element_text(size=16, weight='bold', ha='center'),  # <-- CENTER the title
        axis_title_x=element_text(size=14),
        axis_title_y=element_text(size=14),
        axis_text_x=element_text(size=12),
        axis_text_y=element_text(size=12),
        subplots_adjust={'top': 0.9, 'right': 0.95, 'bottom': 0.15, 'left': 0.28}  # Good margins
    )
)
top10_plot


#**2. Relationship between GDP per Capita and Child Deprivation.**#

"*Does More Money Mean Better Childhoods? The Complex Link Between Wealth and Deprivation*"

**Insight**

Wealth alone does not guarantee lower child deprivation — though a general negative trend exists. Some countries outperform or underperform relative to their income, revealing important policy lessons. Understanding these exceptions could inspire more effective development strategies worldwide.

**Visualization**
Scatter Plot with a Linear Regression Line, plotting GDP per Capita vs. Child Deprivation

In [ ]:
# 📊 Step 2: Create scatterplot with renamed column
from plotnine import ggplot, aes, geom_point, geom_smooth, labs, theme_minimal

scatter_plot = (
    ggplot(deprivation_gdp, aes(x='gdp_per_capita', y='obs_value'))
    + geom_point(color='tomato')
    + geom_smooth(method='lm', se=False, color='black')
    + labs(
        title='Child Deprivation vs GDP per Capita',
        x='GDP per capita (2015 US$)',
        y='Deprivation (%)'
    )
    + theme_minimal()
)
scatter_plot


## **3.  Trends in Female Adolescent Inactivity for Selected Countries**


"*Progress or Plateau? Tracking Girls’ Inactivity Rates Over Time*"

**Insight**

India, Brazil, and the United States show different trajectories in reducing female adolescent inactivity. While some countries steadily improve, others show worrying reversals or stagnation. Long-term patterns suggest that national policies and social movements significantly influence outcomes.

**Visualization**

 Multi-line Time Series Chart showing inactivity over time for India, Brazil, and the United States.



In [ ]:
time_series_plot = (
    ggplot(time_series_data, aes(x='time_period', y='obs_value', color='country'))
    + geom_line() + geom_point()
    + labs(title='Adolescent Inactivity Trends in Selected Countries',
           x='Year', y='Inactivity (%)')
    + theme_minimal()
)
time_series_plot

## **4.Gender Comparison in Adolescent Inactivity**

"*Mind the Gap: How Gender Shapes Physical Inactivity Among Adolescents*"

**Insight**

Across the world, adolescent girls consistently report higher inactivity rates than boys. This persistent gender gap underscores the urgent need for girl-focused sports and health initiatives. Without closing this gap, broader public health goals will remain out of reach.

**Visualization**

Boxplot comparing adolescent inactivity between males and females globally.



In [ ]:
from plotnine import ggplot, aes, geom_boxplot, labs, theme_minimal, scale_fill_manual

# Updated boxplot: Female = Skyblue, Male = Red, Total = Purple
box_plot = (
    ggplot(inactivity_latest_gender, aes(x='sex', y='obs_value', fill='sex'))
    + geom_boxplot()
    + scale_fill_manual(values={
        'Female': 'skyblue',
        'Male': 'red',
        'Total': 'purple'  # mixed color for Total
    })
    + labs(
        title='Comparison of Adolescent Inactivity by Gender and Total',
        x='Gender Category',
        y='Inactivity (%)'
    )
    + theme_minimal()
)

box_plot

#**5. Global Distribution of Child Deprivation (Globe View)**

"*Mapping Child Deprivation Around the Globe*"


**Insight**

Child deprivation is not uniformly distributed — it clusters heavily in regions like Sub-Saharan Africa, South Asia, and parts of Latin America.Some high-income countries show near-zero deprivation, while several low and middle-income countries face extremely high rates. This global view underscores how urgent and geographically targeted interventions are necessary to address global childhood inequalities.

**Visualisation**

3D Rotatable Globe (Plotly Choropleth Globe)



In [ ]:
import plotly.express as px

fig = px.choropleth(
    deprivation_recent,
    locations="alpha_3_code",
    color="obs_value",
    hover_name="country",
    color_continuous_scale="OrRd",
    projection="orthographic",
    title="Children Experiencing Severe Deprivation (Most Recent Data)"
)

fig.update_layout(
    geo=dict(
        showframe=False,
        showcoastlines=True,
        projection_type='orthographic',
        landcolor='lightgray',
        oceancolor='lightblue',
        showocean=True,
        bgcolor='rgba(0,0,0,0)',  # transparent background
    ),
    coloraxis_colorbar=dict(title="Deprivation (%)")
)

fig.show()


## **Conclusion**

This analysis examines global patterns of adolescent physical inactivity and child deprivation, revealing deep inequalities across countries and genders. The top 10 countries with the highest female inactivity rates highlight urgent areas for public health intervention. A scatterplot analysis shows that while wealth generally reduces child deprivation, the relationship is far from perfect. Trends over time for India, Brazil, and the U.S. expose varying national progress in promoting youth health.
Gender comparisons emphasize persistent disparities requiring targeted action. The globe visualization powerfully illustrates how child deprivation remains heavily regionalized, urging collective global efforts for more inclusive development.